In [154]:
import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

print(plotly.__version__) # requires version >= 1.9.0
init_notebook_mode(connected=True)

2.2.2


In [155]:
import pandas as pd
import cufflinks as cf
cf.set_config_file(offline=True, world_readable=True, theme='ggplot')

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [156]:
df = pd.read_csv('cbo.csv')
df

,Income,2019,2021,2023,2025,2027
0,0 - 10k,1540,5870,7440,8680,10070
1,10k - 20k,960,9050,11400,12180,16060
2,20k - 30k,80,9000,10200,12210,16720
3,30k - 40k,-3920,770,2440,2560,7610
4,40k - 50k,-6040,-2660,-1800,-1530,5270
5,50k - 75k,-22270,-19470,-16940,-17380,3980
6,75k - 100k,-21520,-21260,-18470,-19540,-1390
7,100k - 200k,-64240,-63990,-52900,-55470,-5340
8,200k - 500k,-59570,-60110,-50010,-54530,-5190
9,500k - 1M,-24880,-24080,-18690,-20000,-1940


In [157]:
df.columns[1:]

Index(['2019', '2021', '2023', '2025', '2027'], dtype='object')

In [158]:
list(df['Income'])

['0 - 10k',
 '10k - 20k',
 '20k - 30k',
 '30k - 40k',
 '40k - 50k',
 '50k - 75k',
 '75k - 100k',
 '100k - 200k',
 '200k - 500k',
 '500k - 1M',
 '1M and above',
 'All']

In [159]:
df2 = pd.DataFrame(data=df.iloc[:,1:].transpose(), index=df.columns[1:])
df2 = df2/1000 # convert millions USD to billions USD
df2.columns=list(df['Income']) # like a transpose?
df2

,0 - 10k,10k - 20k,20k - 30k,30k - 40k,40k - 50k,50k - 75k,75k - 100k,100k - 200k,200k - 500k,500k - 1M,1M and above,All
2019,1.54,0.96,0.08,-3.92,-6.04,-22.27,-21.52,-64.24,-59.57,-24.88,-34.10,-233.95
2021,5.87,9.05,9.00,0.77,-2.66,-19.47,-21.26,-63.99,-60.11,-24.08,-28.69,-195.57
2023,7.44,11.40,10.20,2.44,-1.80,-16.94,-18.47,-52.90,-50.01,-18.69,-13.10,-140.40
2025,8.68,12.18,12.21,2.56,-1.53,-17.38,-19.54,-55.47,-54.53,-20.00,-15.81,-148.62
2027,10.07,16.06,16.72,7.61,5.27,3.98,-1.39,-5.34,-5.19,-1.94,-5.78,40.11


In [160]:
# cf.getThemes()
# cf.colors.scales()

In [161]:
#cf.set_config_file(offline=True, world_readable=True, theme='pearl')
iplot([ go.Heatmap(z=df2,
                   x=df2.index,
                   y=df2.columns)])
# iplot([{
#     'x': df2.index,
#     'y': df[col],
#     'name': col
# }  for col in df.columns], filename='cufflinks/simple-line')
# df2.iplot(kind='heatmap', colorscale='puor')

TypeError: Object of type 'DataFrame' is not JSON serializable